In [5]:
# Load modules
import os
import pandas as pd
import re
import ast
import glob
from pathlib import Path

path = "C:/Users/19049/coding_files/data/CFM5.3_traces.ts"
subset = "C:/Users/19049/coding_files/data/subset.ts"
folder = "C:/Users/19049/coding_files/faults/"

to_open = subset

#currently the path is saevd as a .ts file, it'd be nice to have a conversion automatically from utm

In [6]:
#I did not write this https://stackoverflow.com/questions/185936/how-to-delete-the-contents-of-a-folder
#Just a tool to clear the faults folder- but could be useful later when reading in new data
def Delete_faults_content():
    [f.unlink() for f in Path("C:/Users/19049/coding_files/faults").glob("*") if f.is_file()]
Delete_faults_content()

In [7]:
# @title
#what i've got:
#strip the crap from the fault name line (the tag and excess quotes, can expand to more stuff)
#open a fault folder to stick the coords in

#What i want:
#how tf do I put the coords into the new files now- I think I can use the current name and use the make_new_fault as a ticker, so the "put number in folder" function is dumb and every time it sees
#a coord it will just add it to the fault that is currently "name" and that should change since the for loop goes down the list in order
#it'd be nice to get a counter as a discrepancy check

#strips "> " and extra "" from line and sets it to global name
def redefine_name():
    global name
    t1_name = line
    t2_name = re.sub("[> ]", "", t1_name)
    name = ast.literal_eval(t2_name)

#makes an empty pandas dataframe using the empty list l
def make_new_fault():
    global l1
    l1 = pd.DataFrame(l,columns = ['Easting(m)','Northing(m)','Elevation(m)'])

#the line being read is delimited and turned from a line of strings into a line of floats
#then it is apended to the l1 dataframe
def number_line():
    string_line = line.split()
    float_list = list(map(float, string_line))
    l1.loc[len(l1)] = float_list

#takes the finished l1 dataframe and names it the current fault trace, making it a csv    
def save_line():
    l1.to_csv(folder+name+".csv")


In [8]:
#l being here resets the dataframe every time it's run, though I can probably just define make_new_fault pd.dataframe([],columns....) instead of making l=[]
l = []
with open(to_open, encoding='utf-8') as f:
    for line in f:
        if line.startswith("> "):
            is_number = False
            redefine_name()
            make_new_fault()
        elif line.startswith("#"):
            print("doin nothing")
        else:
            is_number = True
            while is_number == True:
                if not line.startswith("> "):
                    number_line()
                break
            save_line()

In [ ]:
#I suppose next we list out the faults we want in a variable or tuple, and we go through the files in fault folder, and whenever the name satisfies the condition
#we do a thing, like open the file, adjust the origin, then save as another adjusted file in a different folder

In [9]:
want = ['Panamint_Valley','Southern_Death_Valley','Black_Mountain','Owl_Lake','Hunter_Mountain','Ash_Hill','Independence_Sierra_Nevada','Independence','Little_Lake','Airport_Lake','Eastern_Little_Lake_north_north_east_cross','Owens_Valley','Northern_Death_Valley']
print(want[5])

Ash_Hill


In [10]:
names = os.listdir(r"C:\Users\19049\coding_files\faults")
names

['.ipynb_checkpoints',
 'BNRA-BMFZ-MULT-Black_Mountain_fault-CFM3.csv',
 'BNRA-NDVZ-MULT-Northern_Death_Valley_fault-CFM2.csv',
 'BNRA-SDVZ-MULT-Southern_Death_Valley_fault_vertical-CFM3.csv']

In [ ]:
#def check_name(thing):
    

In [14]:
for each_fault in want:
    if each_fault in names.__contains__(want[:]):
        print('yes')
    else:
        print('no')
    print(each_fault)

TypeError: argument of type 'bool' is not iterable

In [34]:
glob.glob(r"C:\Users\19049\coding_files\faults\*")

['C:\\Users\\19049\\coding_files\\faults\\BNRA-BMFZ-MULT-Black_Mountain_fault-CFM3.csv',
 'C:\\Users\\19049\\coding_files\\faults\\BNRA-NDVZ-MULT-Northern_Death_Valley_fault-CFM2.csv',
 'C:\\Users\\19049\\coding_files\\faults\\BNRA-SDVZ-MULT-Southern_Death_Valley_fault_vertical-CFM3.csv']

In [36]:
print(*Path(r"C:\Users\19049\coding_files\faults").iterdir(), sep="\n")

C:\Users\19049\coding_files\faults\.ipynb_checkpoints
C:\Users\19049\coding_files\faults\BNRA-BMFZ-MULT-Black_Mountain_fault-CFM3.csv
C:\Users\19049\coding_files\faults\BNRA-NDVZ-MULT-Northern_Death_Valley_fault-CFM2.csv
C:\Users\19049\coding_files\faults\BNRA-SDVZ-MULT-Southern_Death_Valley_fault_vertical-CFM3.csv


In [40]:
for path in Path(r'../coding_files/faults').iterdir():
    print(path)

..\coding_files\faults\.ipynb_checkpoints
..\coding_files\faults\BNRA-BMFZ-MULT-Black_Mountain_fault-CFM3.csv
..\coding_files\faults\BNRA-NDVZ-MULT-Northern_Death_Valley_fault-CFM2.csv
..\coding_files\faults\BNRA-SDVZ-MULT-Southern_Death_Valley_fault_vertical-CFM3.csv


In [48]:
contents = list(Path(r'../coding_files/faults').iterdir())
print(contents, sep="\n")

[WindowsPath('../coding_files/faults/.ipynb_checkpoints'), WindowsPath('../coding_files/faults/BNRA-BMFZ-MULT-Black_Mountain_fault-CFM3.csv'), WindowsPath('../coding_files/faults/BNRA-NDVZ-MULT-Northern_Death_Valley_fault-CFM2.csv'), WindowsPath('../coding_files/faults/BNRA-SDVZ-MULT-Southern_Death_Valley_fault_vertical-CFM3.csv')]
